In [1]:
%matplotlib inline

합성곱 신경망
==========================================================================


# 데이터 셋 사전 정의

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

Using cuda device


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



# 데이터 셋에서 샘플 데이터 가져오기

In [3]:
x,y = next(iter(train_dataloader))
x.shape

torch.Size([64, 1, 28, 28])

# 합성곱 레이어 (Convolutional Layer)

In [4]:
conv_layer = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)
output = conv_layer(x)
output.shape

torch.Size([64, 6, 24, 24])

# 풀링 레이어 (Pooling Layer)

In [5]:
max_pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)
output = max_pool_layer(output)
output.shape

torch.Size([64, 6, 12, 12])

# 활성 레이어 (Activation Layer)
- ReLU

In [6]:
activation_layer = nn.ReLU()
output = activation_layer(output)
output.shape

torch.Size([64, 6, 12, 12])

# 2번째 Conv layer 및 pooling, activation layer

In [7]:
conv_layer = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
output = conv_layer(output)
print(output.shape)

max_pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)
output = max_pool_layer(output)
print(output.shape)

activation_layer = nn.ReLU()
output = activation_layer(output)
print(output.shape)

torch.Size([64, 16, 8, 8])
torch.Size([64, 16, 4, 4])
torch.Size([64, 16, 4, 4])


신경망(Neural Networks)
=======================

신경망은 ``torch.nn`` 패키지를 사용하여 생성할 수 있습니다. \
우리는 지난 시간까지 배운 신경망 구조는 2개의 입력, 출력 레이어로 구성되어 있고 1개의 은닉(hidden layer)레이어로 구성되어 있는 \
MLP 신경망 (Multi-Layer Perceptron)의 구조를 가지고 학습을 진행했습니다.

MLP신경망과 같은 신경망을 순전파 네트워크(Feed-forward network)라고 합니다. \
입력(input)을 받아 여러 계층에 차례로 전달한 후, 최종 출력(output)을 제공합니다.

모델의 클래스를 정의하는데 pytorch의 ``nn.Module``을 상속받아 사용했습니다.\
``nn.Module`` 은 모델 구조를 정의하는 계층(layer)과 ``output`` 을 반환하는 신경망 내의 연산 등을 정의하는 ``forward(input)``
메서드를 포함하고 있습니다.

복습차원에서 신경망의 일반적인 학습 과정을 정리하자면 다음과 같습니다:

- 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.
- 데이터셋(dataset) 입력을 반복합니다.
- 입력을 신경망에서 전파(process)합니다.
- 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산합니다.
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.
- 신경망의 가중치를 갱신합니다. 일반적으로 다음과 같은 간단한 규칙을 사용합니다:
  ``새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)``

흑백 숫자 이미지 데이터 셋인 ``MNIST``을 사용해 이미지를 분류하는 신경망을 예제로 살펴보겠습니다.

아래 이미지는 딥러닝의 신경망 중 대표적인 ``Convolution Neural Network`` 입니다.

![convnet](../_static/mnist.png)


신경망 정의하기
------------------

이제 신경망을 정의해보겠습니다:

In [24]:
import torch
import torch.nn as nn

class LeNet5(nn.Module):

    def __init__(self):
        super(LeNet5, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 24, 24, 6)
        #    Pool     -> (?, 12, 12, 6)
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 ImgIn shape=(?, 12, 12, 16)
        #    Conv      ->(?, 8, 8, 16)
        #    Pool      ->(?, 4, 4, 16)
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        # L3 FC 4x4x16 inputs -> 120 outputs
        self.fc1 = nn.Linear(4 * 4 * 16, 120)
        self.layer3 = nn.Sequential(
            self.fc1,
            nn.ReLU())
        
        # L3 FC 120 inputs -> 84 outputs
        self.fc2 = nn.Linear(120, 84)
        self.layer4 = nn.Sequential(
            self.fc2,
            nn.ReLU())
        
        # L5 Final FC 84 inputs -> 10 outputs
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.fc3(out)
        return out

model = LeNet5().to(device)
print(model)

LeNet5(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (layer3): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): ReLU()
  )
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (layer4): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
  )
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# Data Flow
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> Loss
      -> loss

하이퍼파라매터(Hyperparameter)
------------------------------------------------------------------------------------------

하이퍼파라매터(Hyperparameter)는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수입니다.\
서로 다른 하이퍼파라매터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있습니다.\
(하이퍼파라매터 튜닝(tuning)에 대해 [더 알아보기](https://tutorials.pytorch.kr/beginner/hyperparameter_tuning_tutorial.html) )

학습 시에는 다음과 같은 하이퍼파라매터를 정의합니다:
 - **에폭(epoch) 수** - 데이터셋을 반복하는 횟수
 - **배치 크기(batch size)** - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
 - **학습률(learning rate)** - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.
 - **반복(Iteration)** - 한 에폭을 배치 크기로 수행하는데 걸린 횟수

위 하이퍼 파라미터 중 배치크기, 에폭, 반복을 그림으로 나타내면 다음과 같습니다.

![Batch_Epoch_Iter](../_static/batch_epoch_iteration_pic.png)


In [25]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

최적화 단계(Optimization Loop)
------------------------------------------------------------------------------------------

하이퍼파라매터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화할 수 있습니다.
최적화 단계의 각 반복(iteration)을 **에폭** 이라고 부릅니다.

하나의 에폭은 다음 두 부분으로 구성됩니다:
 - **학습 단계(train loop)** - 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴합니다.
 - **검증/테스트 단계(validation/test loop)** - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)합니다.

학습 단계(training loop)에서 일어나는 몇 가지 개념들을 간략히 살펴보겠습니다. 최적화 단계(optimization loop)를 보려면
`full-impl-label` 부분으로 건너뛰시면 됩니다.


손실 함수(loss function)
------------------------------------------------------------------------------------------

학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높습니다.
**손실 함수(loss function)**\ 는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 합니다. \
주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산합니다.

일반적인 손실함수로는 \
회귀 문제(regression task)에 사용하는 [`nn.MSELoss`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss>) (평균 제곱 오차(MSE; Mean Square Error), \
분류 문제(classification task)에 사용하는 [`nn.NLLLoss`](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (음의 로그 우도(Negative Log Likelihood)), \
그리고 ``nn.LogSoftmax`` 와 ``nn.NLLLoss`` 를 합친 [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)등이 있습니다.

이번 수업에서는 모델의 출력 로짓(logit)을 ``nn.CrossEntropyLoss`` 에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산합니다.



In [26]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

옵티마이저(Optimizer)
------------------------------------------------------------------------------------------

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다. **최적화 알고리즘** 은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD; Stochastic Gradient Descent))을 정의합니다. \
모든 최적화 절차(logic)는 ``optimizer`` 객체에 캡슐화(encapsulate)됩니다. 여기서는 SGD 옵티마이저를 사용하고 있으며, PyTorch에는 ADAM이나 RMSProp과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는
[다양한 옵티마이저](https://pytorch.org/docs/stable/optim.html) 가 있습니다.

학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라매터를 등록하여 옵티마이저를 초기화합니다.



In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

학습 단계(loop)에서 최적화는 세단계로 이뤄집니다:
 * ``optimizer.zero_grad()`` 를 호출하여 모델 매개변수의 변화도를 재설정합니다. \
 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정합니다.


 * ``loss.backwards()`` 를 호출하여 예측 손실(prediction loss)을 역전파합니다. \
 앞서 autograd에서 배웠듯이 PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.
 
 * 변화도를 계산한 뒤에는 ``optimizer.step()`` 을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.




전체 구현
------------------------------------------------------------------------------------------

최적화 코드를 반복하여 수행하는 ``train_loop`` 와 테스트 데이터로 모델의 성능을 측정하는 ``test_loop`` 를 정의하였습니다.



In [28]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실 함수와 옵티마이저를 초기화하고 ``train_loop`` 와 ``test_loop`` 에 전달합니다.
모델의 성능 향상을 알아보기 위해 자유롭게 에폭(epoch) 수를 증가시켜 볼 수 있습니다.



In [29]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299852  [    0/60000]
loss: 2.296583  [ 6400/60000]
loss: 2.324379  [12800/60000]
loss: 2.289900  [19200/60000]
loss: 2.311976  [25600/60000]
loss: 2.304573  [32000/60000]
loss: 2.299873  [38400/60000]
loss: 2.313729  [44800/60000]
loss: 2.299645  [51200/60000]
loss: 2.292958  [57600/60000]


KeyboardInterrupt: ignored

------------------------------------------------------------------------------------------




#### Q1. CNN Layer 3개로 수정하기

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=?, stride=?, padding=?),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=?, stride=?, padding=?),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=?, stride=?, padding=?),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU())
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out